In [1]:
import argparse
from datetime import datetime
import json
import logging
import os
import math
import sys
import time


from gridappsd import GridAPPSD, DifferenceBuilder, utils, topics as t
from gridappsd.topics import simulation_input_topic, simulation_log_topic, simulation_output_topic, platform_log_topic


#model_mrid = sim_request["power_system_config"]["Line_name"]
#model_mrid = "_C125761E-9C21-4CA9-9271-B168150DE276" #ieee13training
#model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node
#model_mrid = "_AAE94E4A-2465-6F5E-37B1-3E72183A4E44" #test9500
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" #ieee13assets
#model_mrid="_C1C3E687-6FFD-C753-582B-632A27E28507" # IEEE 123
#model_mrid="_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"

requestedTime=1570041125



global BaseConnDict,BaseTermDict,TermList, NodeList
global XfmrKeys,XfmrDict,SwitchKeys,SwitchDict,DG_query

os.environ['GRIDAPPSD_APPLICATION_ID'] = 'gridappsd-topology-processor'
os.environ['GRIDAPPSD_APPLICATION_STATUS'] = 'STARTED'
#os.environ['GRIDAPPSD_SIMULATION_ID'] = opts.simulation_id
os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'


gapps = GridAPPSD()
assert gapps.connected

In [ ]:
from topologyprocessor import TopologyProcessor
from topologydictionary_broken import TopologyDictionary
from networkmodel import NetworkModel
from distributedtopology import DistributedTopology

In [ ]:
model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node

In [ ]:
Topology = LocalTopologyDictionary(gapps, model_mrid)
network = NetworkModel(gapps)
network.build_equip_dicts(model_mrid, Topology)
EqTypes = ['ACLineSegment', 'PowerTransformer', 'TransformerTank', 'SynchronousMachine']
Topology.build_linknet(EqTypes)


In [ ]:
Topology.update_switches()

In [ ]:
Topology.build_feeder_islands()

In [ ]:
FirstNode

In [2]:
message = {
   "requestType": "GET_SWITCH_AREAS",
   "modelID": "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
   "resultFormat": "JSON"
}

topic = "goss.gridappsd.request.data.topology"

gapps.get_response(topic, message, timeout=30)

{'feeders': {'_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62': {'addressable_equipment': ['_52DE9189-20DC-4C73-BDEE-E960FE1F9493',
    '_CE5D0651-676B-4AF3-8D67-41BF1B33E30C',
    '_2858B6C2-0886-4269-884C-06FA8B887319',
    '_517413CB-6977-46FA-8911-C82332E42884',
    '_CE5D0651-676B-4AF3-8D67-41BF1B33E30C',
    '_517413CB-6977-46FA-8911-C82332E42884',
    '_2858B6C2-0886-4269-884C-06FA8B887319'],
   'unaddressable_equipment': ['_44FC5A86-A099-45B8-B847-F685C5027AFB',
    '_B6363F07-B1BC-420B-AA4C-A34BB8F05827',
    '_E2E0FC64-8D45-4C55-BDB9-EAB827A46FBC',
    '_7c6f94c1-1419-4582-ab2d-a0b11772c26b',
    '_b393e719-0981-4498-9d96-07f1be7de009',
    '_f11a9ad9-5b68-483b-b52f-dd4af07bb77d',
    '_44FC5A86-A099-45B8-B847-F685C5027AFB',
    '_B6363F07-B1BC-420B-AA4C-A34BB8F05827',
    '_E2E0FC64-8D45-4C55-BDB9-EAB827A46FBC',
    '_A04CDFB1-E951-4FC4-8882-0323CD70AE3C',
    '_44FC5A86-A099-45B8-B847-F685C5027AFB',
    '_B6363F07-B1BC-420B-AA4C-A34BB8F05827',
    '_E2E0FC64-8D45-4C55-BDB9-EAB827A46F

In [ ]:
Topology.EquipDict['SynchronousMachine']

In [ ]:
Topology.ConnNodeDict['_E3096E09-7BE0-451A-A3B8-8C3F353931B8']

In [ ]:
Topology.ConnNodeDict[Topology.NodeList[5288]]

In [ ]:
Topology.TerminalsDict[Topology.TermList[10433]]

In [ ]:
Topology.ConnNodeDict[Topology.TerminalsDict[Topology.TermList[10433]]['ConnectivityNode']]

In [ ]:
message = {
   "requestType": "GET_BASE_Topology",
   "modelID": "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
   "resultFormat": "JSON"
}

topic = "goss.gridappsd.request.data.topology"

gapps.get_response(topic, message, timeout=30)

In [ ]:
import topo_meas_queries as queries
queries.get_all_lines(gapps, model_mrid)

In [ ]:
from gridappsd import topics as t
topic = t.REQUEST_POWERGRID_DATA

object_mrid = '_62DDC18B-DFF3-4DE5-AA1B-97AE04720311'

message = {
    "requestType": "QUERY_OBJECT_DICT",
    "resultFormat": "JSON",
    "modelId": model_mrid,
    "objectId": object_mrid
}

data = gapps.get_response(topic, message)
data['data']

In [ ]:
Topology.build_feeder_islands()

In [ ]:
Topology.EquipDict['PowerTransformer'].keys()

In [ ]:
DistTopo = DistributedTopology(gapps, model_mrid)
message = DistTopo.create_switch_areas(model_mrid)

In [ ]:
import time, json

class LocalTopologyDictionary():
    
    def __init__(self, gapps, model_mrid):
        self.model_mrid = model_mrid
        self.gapps = gapps
        self.EquipDict = {}
        self.ConnNodeDict = {}
        self.TerminalsDict = {}
        self.NodeList = []
        self.TermList = []
        self.Feeders = {}
        self.Islands = {}
        Tree = {}
        
    # Builds LinkNet linked lists for all CIM classes specified by EqTypes
    def build_linknet(self, EqTypes):
        # Intialize counter objects
        index = 0
        counter = 0
        # Build LinkNetList for all specified CIM classes:
        for i0 in range(len(EqTypes)): 
            [index, counter] = self.build_class_lists(EqTypes[i0], index, counter)
        # Add floating nodes not connected to a branch:
        StartTime = time.process_time()
        AllNodes = list(self.ConnNodeDict.keys())
        MissingNodes = list(set(AllNodes).difference(self.NodeList))
        for i1 in range(len(MissingNodes)):
            node = MissingNodes[i1]
            if 'list' not in self.ConnNodeDict[node]:
                self.ConnNodeDict[node]['node'] = index+1
                self.ConnNodeDict[node]['list'] = 0
                index = index+1
                self.NodeList.append(node)
        print("Processed ", len(MissingNodes), "missing nodes in "+ str(round(1000*(time.process_time() - StartTime))) + " ms")
        # Dump JSON copies of base LinkNet structure. These are used to rebuild topo after each switch change
        self.BaseConnDict = json.dumps(self.ConnNodeDict)
        self.BaseTermDict = json.dumps(self.TerminalsDict)


    # Build LinkNet structure for single CIM equipment type, called by build_linknet()
    # Three-winding transformers not yet supported
    def build_class_lists(self, eqtype, index, old_counter):
        i2 = -1
        StartTime = time.process_time()
        EquipKeys = list(self.EquipDict[eqtype])

        for i2 in range(len(EquipKeys)):
            # Identify nodes and terminals for readability
            term1=self.EquipDict[eqtype][EquipKeys[i2]]['term1']
            node1=self.EquipDict[eqtype][EquipKeys[i2]]['node1']
            # Create keys for new terminals
            self.TerminalsDict[term1] = {}
            self.TerminalsDict[term1]['ConnectivityNode'] = node1
            self.TermList.append(term1)
            # If node1 not in LinkNet , create new keys
            if 'node' not in self.ConnNodeDict[node1]:
                self.ConnNodeDict[node1]['node'] = index+1
                self.ConnNodeDict[node1]['list'] = 0
                index = index+1
                self.NodeList.append(node1)

            # If two-terminal device, process both terminals
            if 'node2' in self.EquipDict[eqtype][EquipKeys[i2]]:
                # Identify nodes and terminals for readability
                term2=self.EquipDict[eqtype][EquipKeys[i2]]['term2']
                node2=self.EquipDict[eqtype][EquipKeys[i2]]['node2']
                # Create keys for new terminals
                self.TerminalsDict[term2] = {}
                self.TerminalsDict[term2]['ConnectivityNode'] = node2
                self.TerminalsDict[term1]['term'] = 2*(i2+old_counter)+1
                self.TerminalsDict[term2]['term'] = 2*(i2+old_counter)+2
                self.TermList.append(term2)
                # If node2 not in LinkNet , create new keys
                if 'node' not in self.ConnNodeDict[node2]: 
                    self.ConnNodeDict[node2]['node'] = index+1
                    self.ConnNodeDict[node2]['list'] = 0
                    index = index+1
                    self.NodeList.append(node2)
                # 1. Move node list variables to terinal next    
                self.TerminalsDict[term1]['next'] = self.ConnNodeDict[node1]['list']
                self.TerminalsDict[term2]['next'] = self.ConnNodeDict[node2]['list']
                # 2. Populate Terminal list far field with nodes
                self.TerminalsDict[term1]['far'] = self.ConnNodeDict[node2]['node']
                self.TerminalsDict[term2]['far'] = self.ConnNodeDict[node1]['node']
                # 3. Populate Connectivity nodes list with terminals
                self.ConnNodeDict[node1]['list'] = self.TerminalsDict[term1]['term']
                self.ConnNodeDict[node2]['list'] = self.TerminalsDict[term2]['term']

            # If one-terminal device, process only single terminal
            else:
                self.TerminalsDict[term1]['term'] = i2+2*(old_counter)+1
                self.TerminalsDict[term1]['next'] = 0
                self.TerminalsDict[term1]['far'] = 0
                self.ConnNodeDict[node1]['list'] = self.TerminalsDict[term1]['term']

        print("Processed " + str(i2+1) + str(eqtype) + "objects in " + str(round(1000*(time.process_time() - StartTime))) + " ms")

        counter = old_counter+i2+1
        return index, counter
    

    # Method to update Linknet Lists with current switch positions
    def update_switches(self):

        SwitchKeys = list(self.EquipDict['Breaker'].keys()) + list(self.EquipDict['Fuse'].keys()) + list(self.EquipDict['LoadBreakSwitch'].keys()) + list(self.EquipDict['Recloser'].keys())
        SwitchDict = {}
        SwitchDict.update(self.EquipDict['Breaker'])
        SwitchDict.update(self.EquipDict['Fuse'])
        SwitchDict.update(self.EquipDict['LoadBreakSwitch'])
        SwitchDict.update(self.EquipDict['Recloser'])

        self.ConnNodeDict = json.loads(self.BaseConnDict)
        self.TerminalsDict = json.loads(self.BaseTermDict)

        StartTime = time.process_time()

        for i3 in range(len(SwitchKeys)):

            node1=SwitchDict[SwitchKeys[i3]]['node1']
            node2=SwitchDict[SwitchKeys[i3]]['node2']


            # If switch closed, merge nodes
            if SwitchDict[SwitchKeys[i3]]['open'] == 1:
                # Merge topology Nodes
                #ConnNodeDict[node1]['TopologicalNode'] = tpnode1
                self.ConnNodeDict[node2]['TopologicalNode'] = self.ConnNodeDict[node1]['TopologicalNode'] #tpnode1
                #TopoNodeDict[tpnode1] = [node1, node2] # not implemented
                #TopoNodeDict[tpnode2] = [node2, node1]

                # Update Linked Lists
                if self.ConnNodeDict[node2]['list'] > self.ConnNodeDict[node1]['list']:
                    term2 = self.TermList[self.ConnNodeDict[node2]['list']-1]
                    next2 = self.TerminalsDict[term2]['next']
                    while next2 != 0:
                        term2 = self.TermList[next2-1]
                        next2 = self.TerminalsDict[term2]['next']
                    self.TerminalsDict[term2]['next'] = self.ConnNodeDict[node1]['list']
                    self.ConnNodeDict[node1]['list'] = self.ConnNodeDict[node2]['list']
                else:
                    term1 = self.TermList[self.ConnNodeDict[node1]['list']-1]
                    next1 = self.TerminalsDict[term1]['next']
                    while next1 != 0:
                        term1 = self.TermList[next1-1]
                        next1 = self.TerminalsDict[term1]['next']
                    self.TerminalsDict[term1]['next'] = self.ConnNodeDict[node2]['list']
                    self.ConnNodeDict[node2]['list'] = self.ConnNodeDict[node1]['list']

        print("Processed " + str(i3+1) + "switch objects in " + str(round(1000*(time.process_time() - StartTime))) + " ms")

    def build_feeder_islands(self):
        Tree = {}
        fdr = -1
        isl = -1
        
        # Iterate through all PowerTransfomer objects
        XfmrKeys = list(self.EquipDict['PowerTransformer'].keys())
        print(XfmrKeys)
        StartTime = time.process_time()
        for i4 in range(len(XfmrKeys)):
            SubXfmr = XfmrKeys[i4]
            # Identify substation transfomers with high-side >34kV and low-side >1kV
            volt1 = self.EquipDict['PowerTransformer'][SubXfmr]['volt1']
            volt2 = self.EquipDict['PowerTransformer'][SubXfmr]['volt2']
            if volt1 >= 34000 and 34000 >= volt2 >= 1000:
                fdr = fdr + 1
                print(SubXfmr)
                self.spanning_tree('PowerTransformer', [SubXfmr], Tree, 'single') 
                 # Add nodes to Feeder dictionary
                self.Feeders['feeder_' + str(fdr)] = {}
                self.Feeders['feeder_' + str(fdr)]['sub_xfmr'] = SubXfmr
                self.Feeders['feeder_' + str(fdr)]['connectivity_node'] = Tree[SubXfmr] 
                # Add feeder to Node dictionary
                for i5 in range(len(Tree[SubXfmr])): 
                    self.ConnNodeDict[Tree[SubXfmr][i5]]['Feeder'].append(('feeder_' + str(fdr)))
        print('Processed ' + str(fdr + 1) + ' feeders in ' + str(round(1000*(time.process_time() - StartTime))) + " ms")
        
        # Iterate through all SynchronousMachine objects
        StartTime = time.process_time()
        DGKeys = list(self.EquipDict['SynchronousMachine'].keys())
        for i6 in range(len(DGKeys)):
            DG = DGKeys[i6]
            DGNode = self.EquipDict['SynchronousMachine'][DG]['node1']
            [not_in_feeder, found] = self.check_tree(DGNode, self.Feeders, 'all', DG)
            if not_in_feeder:
                [not_in_island, found] = self.check_tree(DGNode, self.Islands, 'all', DG)
                if not_in_island:
                    isl = isl + 1
                    self.spanning_tree('SynchronousMachine', [DG], Tree, 'single')
                    self.Islands['island_' + str(isl)] = {}
                    self.Islands['island_' + str(isl)]['SynchronousMachine'] = [DG]
                    # Add island to Node dictionary
                    for i7 in range(len(Tree[DG])): 
                        self.ConnNodeDict[Tree[DG][i7]]['Island'].append(('island_' + str(isl)))
                else:
                    self.Islands['island_' + str(isl)]['SynchronousMachine'].append(DGKeys[i6])
        print('Processed ' + str(isl + 1) + ' islands in ' + str(round(1000*(time.process_time() - StartTime))) + " ms")
        
    
    def spanning_tree(self, eqtype, RootKeys, Tree, Scope):

        TotalNodes=0
        old_len = len(Tree.keys())
        StartTime = time.process_time()

        
        for i6 in range(len(RootKeys)):
            root = RootKeys[i6]
            Tree[root] = []

            # If switch object, only use second node
            if eqtype in ['Breaker', 'Fuse', 'LoadBreakSwitch', 'Recloser']:
                Tree[root].append(self.EquipDict[eqtype][root]['node2'])
                FirstNode = 0
                LastNode = 1 # only 1 node used, so initialize list at 0,1
            # If DER object, only has one node
            elif eqtype in ['SynchronousMachine', 'PowerElectronicsConnection']:
                #[not_in_tree, found] = self.check_tree(self.EquipDict[eqtype][root]['node1'], Tree, Scope, root)
               # if not_in_tree:
                Tree[root].append(self.EquipDict[eqtype][root]['node1'])
                FirstNode = 0 
                LastNode = 1 # only 1 node exists, so initialize list at 0,1
                #else:
                    
            # Otherwise, use both nodes    
            else: # Then 2-terminal object
                print(root)
                print(eqtype)
                [not_in_tree, found] = self.check_tree(self.EquipDict[eqtype][root]['node2'], Tree, Scope, root)
                if not_in_tree:
                    Tree[root].append(self.EquipDict[eqtype][root]['node1'])
                    Tree[root].append(self.EquipDict[eqtype][root]['node2'])
                    FirstNode = 1 
                    LastNode = 2 # 2 nodes in starting list, so initialize at 1,2
                else:
                    break
            while LastNode != FirstNode:
                NextTerm = self.ConnNodeDict[Tree[root][FirstNode]]['list']
                FirstNode = FirstNode + 1
                while NextTerm != 0:
                    # Get next node and terminal for current node
                    NextNode = self.TerminalsDict[self.TermList[NextTerm-1]]['far']
                    NextTerm = self.TerminalsDict[self.TermList[NextTerm-1]]['next']
                    node = self.NodeList[NextNode-1]
                    [not_in_tree, found] = self.check_tree(node, Tree, Scope, root)
                    # Add node if not in another tree        
                    if not_in_tree:       
                        if self.ConnNodeDict[node]['nominalVoltage']:
                            # Stop building tree into sub-transmission network
                            if int(self.ConnNodeDict[node]['nominalVoltage']) < 34000: 
                                Tree[root].append(self.NodeList[NextNode-1])
                                LastNode = LastNode + 1                       
                        else: # Add node to tree if no nominal voltage defined
                            Tree[root].append(self.NodeList[NextNode-1])
                            LastNode = LastNode + 1


            print("Processed topology from  " + str(root) + ' with ' + str(len(Tree[root])) + " buses")

        print("Processed " + str(len(Tree.keys()) - old_len) + "topology trees containing " + str(TotalNodes+len(Tree[root])) + " buses in " + str(round(1000*(time.process_time() - StartTime))) + " ms")

        return Tree
    
    # function to check if a node is the spanning tree
    # use argument "all" to check all trees from all root nodes
    # use argument "single" to only check the single tree from current root node
    # node is ConnectivityNode mRID to be checked
    # root is used to specify "single" tree root key 
    def check_tree(self, node, Tree, Scope, root):
        not_in_tree = True
        found = 'False'
        if Scope == 'all': 
            TreeKeys = list(Tree.keys())
            for i7 in range(len(TreeKeys)):
                if node in Tree[TreeKeys[i7]]:
                    not_in_tree = False
                    found = TreeKeys[i7]
                    break
        else: 
            if node in Tree[root]: 
                not_in_tree = False
                found = root
        return not_in_tree, found